In [83]:
import numpy as np
import pandas as pd
from pyomo.opt import SolverFactory
from pyomo.core import Var
import pyomo.environ as en

In [84]:
# input data
# To use this script, the user needs to change the working directory. Or put the data and code in the same folder.
testData = pd.read_csv('testdata_final.csv', parse_dates=['timestamp'], index_col='timestamp')

In [85]:
# define input data
load = testData['actual_consumption'].values
sellPrice = testData['price_sell'].values
buyPrice = testData['price_buy'].values
heat_load = testData['heat_load'].values
cool_load = testData['cool_load'].values
price_gas1 = testData['gas'].values/1000
price_gas = price_gas1*35.3147248277 
# The unit of natural gas price is dollars per thousand cubic feet
# convert into dollers per cubic meter 

In [86]:
# refined PV model, namely Case I. (Users can switch between the two cases at will)
PV_unit = testData['actual_pv'].fillna(0)
PV_unit[PV_unit<0] = 0
PV_unit = PV_unit.values/1000    # The pv output unit in the data is W, converted to kW

In [87]:
# conventional PV model, namely Case II.
# PV_unit = testData['pv_c'].fillna(0).values/1000      

In [88]:
priceDict1 = dict(enumerate(sellPrice))
priceDict2 = dict(enumerate(buyPrice))
priceDict3 = dict(enumerate(price_gas))
LoadDict = dict(enumerate(load))
PVDict = dict(enumerate(PV_unit))
HeatDict = dict(enumerate(heat_load))
CoolDict = dict(enumerate(cool_load))

In [89]:
# create a concrete model
m = en.ConcreteModel()

In [90]:
# define time index
m.Time = en.RangeSet(0,len(load)-1)

In [91]:
# When discussing a Pyomo model, we use the "Parameters" to refer to data that must be provided
# in order to find an optimal (or good) assignment of values to the decision variables.
m.priceSell = en.Param(m.Time, initialize=priceDict1)
m.priceBuy = en.Param(m.Time, initialize=priceDict2)
m.priceGas = en.Param(m.Time, initialize=priceDict3)
m.Load = en.Param(m.Time, initialize=LoadDict)
m.PV_unit = en.Param(m.Time, initialize=PVDict)
m.Heat = en.Param(m.Time,initialize=HeatDict)
m.Cool = en.Param(m.Time,initialize=CoolDict)

In [92]:
# Variables are intended to ultimately be given values by an optimization package. 
# They are declared and optionally bounded, given initial values, and documented using the Pyomo Var function.
m.P_buy = en.Var(m.Time, within=en.Reals)
m.P_sell = en.Var(m.Time, within=en.Reals)

In [93]:
# Define TES parameters
m.TSS_Capacity = en.Param(initialize=10)
m.TSS_Power = en.Param(initialize=10)
m.TSS_Charge_Efficiency = en.Param(initialize=0.95)
m.TSS_Discharge_Efficiency = en.Param(initialize=0.95)
m.TSS_rate = en.Param(initialize=0.04)

In [94]:
# Define heat storage parameters
m.Bool_char_TSS = en.Var(m.Time,within=en.Binary)
m.Bool_dis_TSS = en.Var(m.Time,within=en.Binary,initialize=0)
m.Energy_SOC_TSS = en.Var(m.Time,initialize=0)
m.TSS_P_char = en.Var(m.Time,initialize=0)
m.TSS_P_dis = en.Var(m.Time,initialize=0)
m.n_TSS = en.Var([1],within=en.NonNegativeIntegers)

In [95]:
# the Big M method is a method of solving linear programming problems using the simplex algorithm. 
bigM = 10000000
m.z1_TSS = en.Var(m.Time)
m.z2_TSS = en.Var(m.Time)

In [96]:
# Introduce auxiliary variable 1
def Bool_char_TSS_rule_1(m,i):
    return m.z1_TSS[i] <= bigM*m.Bool_char_TSS[i]
m.TSS_char_cons1 = en.Constraint(m.Time, rule=Bool_char_TSS_rule_1)

def Bool_char_TSS_rule_2(m,i):
    return -bigM*m.Bool_char_TSS[i] <= m.z1_TSS[i]
m.TSS_char_cons2 = en.Constraint(m.Time, rule=Bool_char_TSS_rule_2)

def Bool_char_TSS_rule_3(m,i):
    return m.z1_TSS[i]-m.n_TSS[1] <= bigM*(1-m.Bool_char_TSS[i])
m.TSS_char_cons3 = en.Constraint(m.Time, rule=Bool_char_TSS_rule_3)

def Bool_char_TSS_rule_4(m,i):
    return -bigM*(1-m.Bool_char_TSS[i]) <= m.z1_TSS[1]-m.n_TSS[1]
m.TSS_char_cons4 = en.Constraint(m.Time, rule=Bool_char_TSS_rule_4)

In [97]:
# Introduce auxiliary variable 1
def Bool_dis_TSS_rule_1(m,i):
    return m.z2_TSS[i] <= bigM*m.Bool_dis_TSS[i]
m.TSS_dis_cons1 = en.Constraint(m.Time,rule=Bool_dis_TSS_rule_1)

def Bool_dis_TSS_rule_2(m,i):
    return -bigM*m.Bool_dis_TSS[i] <= m.z2_TSS[i]
m.TSS_dis_cons2 = en.Constraint(m.Time,rule=Bool_dis_TSS_rule_2)

def Bool_dis_TSS_rule_3(m,i):
    return m.z2_TSS[i]-m.n_TSS[1] <= bigM*(1-m.Bool_dis_TSS[i])
m.TSS_dis_cons3 = en.Constraint(m.Time,rule=Bool_dis_TSS_rule_3)

def Bool_dis_TSS_rule_4(m,i):
    return -bigM*(1-m.Bool_dis_TSS[i]) <= m.z2_TSS[i]-m.n_TSS[1]
m.TSS_dis_cons4 = en.Constraint(m.Time,rule=Bool_dis_TSS_rule_4)

In [98]:
# The thermal energy storage power cannot exceed the upper and lower limits
# Nonlinear transformation： m.z2_ESS = m.Bool_char_TSS * m.n_TSS
m.ChargingLimit = en.Param(initialize=m.TSS_Power)

def TSS_char_power_rule_1(m,i):
    return m.TSS_P_char[i] <= m.z1_TSS[i]*m.ChargingLimit
m.TSS_char_power_cons1 = en.Constraint(m.Time,rule=TSS_char_power_rule_1)

def TSS_char_power_rule_2(m,i):
    return 0 <= m.TSS_P_char[i]
m.TSS_char_power_cons2 = en.Constraint(m.Time,rule=TSS_char_power_rule_2)

In [99]:
# The heating power of thermal storage cannot exceed the upper and lower limits
# Nonlinear transformation： m.z2_TSS = m.Bool_dis_TSS * m.n_TSS
m.DischargingLimit = en.Param(initialize=m.TSS_Power)

def TSS_dis_power_rule_1(m,i):
    return m.TSS_P_dis[i] <= m.z2_TSS[i]*m.DischargingLimit
m.TSS_dis_power_cons1 = en.Constraint(m.Time, rule=TSS_dis_power_rule_1)

def TSS_dis_power_rule_2(m,i):
    return 0 <= m.TSS_P_dis[i]
m.TSS_dis_power_cons2 = en.Constraint(m.Time,rule=TSS_dis_power_rule_2)

In [100]:
# TES is not allowed to store and release energy at the same time
def TSS_char_dis(m,i):
    return m.Bool_char_TSS[i]+m.Bool_dis_TSS[i] <= 1
m.TSS_char_dis_cons = en.Constraint(m.Time,rule=TSS_char_dis)

In [101]:
# Heat storage energy
def Energy_SOC_TSS_rule(m,t):
    if t==0:
        return m.Energy_SOC_TSS[t] == 0.2*m.TSS_Capacity*m.n_TSS[1]
    else:
        return m.Energy_SOC_TSS[t] == m.Energy_SOC_TSS[t-1]*(1-m.TSS_rate)+m.TSS_Charge_Efficiency*m.TSS_P_char[t]-m.TSS_P_dis[t]/m.TSS_Discharge_Efficiency                    
m.TSS_Energy_SOC_cons = en.Constraint(m.Time,rule=Energy_SOC_TSS_rule)

def Energy_TSS_rule_1(m,t):
    return m.Energy_SOC_TSS[t] <= 0.9*m.TSS_Capacity*m.n_TSS[1]
m.TSS_Energy_cons1 = en.Constraint(m.Time,rule=Energy_TSS_rule_1)

def Energy_TSS_rule_2(m,t):
    return 0.2*m.TSS_Capacity*m.n_TSS[1] <= m.Energy_SOC_TSS[t]
m.TSS_Energy_cons2 = en.Constraint(m.Time,rule=Energy_TSS_rule_2)

In [102]:
# Number of heat storage installations
def TSS_num_rule(m):
    return m.n_TSS[1] <= 50000
m.TSS_num_cons = en.Constraint(m.Time,rule=TSS_num_rule)

In [103]:
# Define energy storage battery parameters
m.Batt_Capacity = en.Param(initialize=1)
m.Batt_Power = en.Param(initialize=0.075)
m.Batt_Charge_Efficiency = en.Param(initialize=0.95)
m.Batt_Discharge_Efficiency = en.Param(initialize=0.95)
m.Batt_rate = en.Param(initialize=0.04)

In [104]:
# Define energy storage battery parameters
m.Bool_char = en.Var(m.Time,within=en.Binary)
m.Bool_dis = en.Var(m.Time,within=en.Binary,initialize=0)
m.Energy_SOC = en.Var(m.Time,initialize=0)
m.Batt_P_char = en.Var(m.Time,initialize=0)
m.Batt_P_dis = en.Var(m.Time,initialize=0)

m.n_ESS = en.Var([1],within=en.NonNegativeIntegers)

In [105]:
# bigM 
bigM = 10000000
m.z1_Batt = en.Var(m.Time)
m.z2_Batt = en.Var(m.Time)

In [106]:
# Introduce auxiliary variable 1
def Bool_char_rule_1(m,i):
    return( m.z1_Batt[i] <= bigM*m.Bool_char[i] )
m.Batt_char1 = en.Constraint(m.Time,rule=Bool_char_rule_1)

def Bool_char_rule_2(m,i):
    return( -bigM*m.Bool_char[i] <= m.z1_Batt[i] )
m.Batt_char2 = en.Constraint(m.Time,rule=Bool_char_rule_2)

def Bool_char_rule_3(m,i):
    return( m.z1_Batt[i]-m.n_ESS[1] <= bigM*(1-m.Bool_char[i]) )
m.Batt_char3 = en.Constraint(m.Time,rule=Bool_char_rule_3)

def Bool_char_rule_4(m,i):
    return( -bigM*(1-m.Bool_char[i]) <= m.z1_Batt[i]-m.n_ESS[1] )
m.Batt_char4 = en.Constraint(m.Time, rule=Bool_char_rule_4)

In [107]:
# Introduce auxiliary variable 2
def Bool_dis_rule_1(m,i):
    return( m.z2_Batt[i] <= bigM*m.Bool_dis[i] )
m.Batt_dis1 = en.Constraint(m.Time,rule=Bool_dis_rule_1)

def Bool_dis_rule_2(m,i):
    return( -bigM*m.Bool_dis[i] <= m.z2_Batt[i] )
m.Batt_dis2 = en.Constraint(m.Time,rule=Bool_dis_rule_2)

def Bool_dis_rule_3(m,i):
    return( m.z2_Batt[i]-m.n_ESS[1] <= bigM*(1-m.Bool_dis[i]) )
m.Batt_dis3 = en.Constraint(m.Time,rule=Bool_dis_rule_3)

def Bool_dis_rule_4(m,i):
    return( -bigM*(1-m.Bool_dis[i]) <= m.z2_Batt[i]-m.n_ESS[1] )
m.Batt_dis4 = en.Constraint(m.Time,rule=Bool_dis_rule_4)

In [108]:
# The charging power of the electric storage cannot exceed the upper and lower limits
# Nonlinear transformation： m.z1_ESS = m.Bool_char * m.n_ESS
m.ChargingLimit_Batt = en.Param(initialize=m.Batt_Power)

def Batt_char_power_rule_1(m,i):
    return( m.Batt_P_char[i] <= m.z1_Batt[i]*m.ChargingLimit_Batt )
m.Batt_char_power1 = en.Constraint(m.Time,rule=Batt_char_power_rule_1)

def Batt_char_power_rule_2(m,i):
    return( 0 <= m.Batt_P_char[i] )
m.Batt_char_power2 = en.Constraint(m.Time, rule=Batt_char_power_rule_2)

In [109]:
# The discharge power of the electric energy storage cannot exceed the upper and lower limits，
# Nonlinear transformation： m.z2_ESS = m.Bool_dis * m.n_ESS
m.DischargingLimit_Batt = en.Param(initialize=m.Batt_Power)

def Batt_dis_power_rule_1(m,i):
    return( m.Batt_P_dis[i] <= m.z2_Batt[i]*m.DischargingLimit_Batt )
m.Batt_dis_power1 = en.Constraint(m.Time,rule=Batt_dis_power_rule_1)

def Batt_dis_power_rule_2(m,i):
    return( 0 <= m.Batt_P_dis[i] )
m.Batt_dis_power2 = en.Constraint(m.Time, rule=Batt_dis_power_rule_2)

In [110]:
# EES is not allowed to store and release energy at the same time
def Batt_char_dis(m,i):
    return (m.Bool_char[i]+m.Bool_dis[i] <= 1)
m.Batt_char_dis=en.Constraint(m.Time,rule=Batt_char_dis)

In [111]:
# Energy of EES
def Energy_SOC_rule(m,t):
    if t==0:
        return ( m.Energy_SOC[t] == 0.2*m.Batt_Capacity*m.n_ESS[1] )
    else:
        return ( m.Energy_SOC[t] == m.Energy_SOC[t-1]*(1-m.Batt_rate)+m.Batt_Charge_Efficiency*m.Batt_P_char[t]-m.Batt_P_dis[t]/m.Batt_Discharge_Efficiency )
m.Batt_Energy_SOC = en.Constraint(m.Time,rule=Energy_SOC_rule)    

def Energy_ESS_rule_1(m,t):
    return( m.Energy_SOC[t] <= 0.9*m.Batt_Capacity*m.n_ESS[1] )
m.Batt_Energy_ESS1 = en.Constraint(m.Time,rule=Energy_ESS_rule_1)

def Energy_ESS_rule_2(m,t):
    return( 0.2*m.Batt_Capacity*m.n_ESS[1] <= m.Energy_SOC[t] )
m.Batt_Energy_ESS2 = en.Constraint(m.Time,rule=Energy_ESS_rule_2)

In [112]:
# Number of electric energy storage installations
# The number of installations cannot exceed the upper limit
def ESS_num_rule(m):
    return m.n_ESS[1] <= 500000
m.ESS_num_cons = en.Constraint(m.Time, rule=ESS_num_rule)

In [113]:
# electrical chillers
m.n_EC = en.Var([1],within=en.NonNegativeIntegers)
m.P_EC = en.Var(m.Time,within=en.Reals)
m.Q_EC = en.Var(m.Time,within=en.Reals)
m.EC_COP = en.Param(initialize=4)
m.P_EC_Max = en.Param(initialize=200)

In [114]:
# Number of electric chillers installed
# The number of installations cannot exceed the upper limit
def EC_num_rule(m):
    return m.n_EC[1] <= 10000
m.EC_num_cons = en.Constraint(rule=EC_num_rule)

In [115]:
# Output constraint of electric chiller
def P_EC_rule1(m,i):
    return m.P_EC[i] <= m.P_EC_Max*m.n_EC[1]
m.P_EC_cons1 = en.Constraint(m.Time,rule=P_EC_rule1)

In [116]:
# Output constraint of electric chiller
def P_EC_rule2(m,i):
    return 0 <= m.P_EC[i]
m.P_EC_cons2 = en.Constraint(m.Time,rule=P_EC_rule2)

In [117]:
# Electrical chiller is converted to cold energy
def EC_Ele_to_cool_rule(m,i):
    return m.Q_EC[i] == m.P_EC[i]*m.EC_COP
m.EC_ELE_TO_COOL_cons = en.Constraint(m.Time,rule=EC_Ele_to_cool_rule)

In [118]:
# Gas turbine
m.n_GT = en.Var([1],within=en.NonNegativeIntegers)
m.P_GT = en.Var(m.Time,within=en.Reals)
m.GT_Max = en.Param(initialize=400)
m.GT_to_Electricity_Efficient = en.Param(initialize=0.3657)

In [119]:
# Calorific value of natural gas
m.H_gas = en.Param(initialize=9.97)
# Gas turbine consumption
m.Vgas_GT = en.Var(m.Time,within=en.Reals)

In [120]:
# CHP parameters
m.n_CHP = en.Var([1],within=en.NonNegativeIntegers)
m.P_CHP = en.Var(m.Time,within=en.Reals)
m.Q_CHP = en.Var(m.Time,within=en.Reals)
m.CHP_to_Electricity_Efficiency = en.Param(initialize=0.3)
m.CHP_to_Heat_Efficiency = en.Param(initialize=0.45)
m.P_CHP_Max = en.Param(initialize=300)
m.Q_CHP_Max = en.Param(initialize=450)
m.Vgas_CHP = en.Var(m.Time,within=en.Reals)
m.Vgas_CHP_Q = en.Var(m.Time,within=en.Reals)

In [121]:
# AC paramters
m.n_AC = en.Var([1],within=en.NonNegativeIntegers)
m.Q_AC = en.Var(m.Time,within=en.Reals)
m.Vgas_AC = en.Var(m.Time,within=en.Reals)
m.AC_COP = en.Param(initialize=1.3)
m.Q_AC_Max = en.Param(initialize=100)

In [122]:
# Number of AC
# The number of installations cannot exceed the upper limit
# Since AC absorbs the waste heat of CHP unit, when CHP unit is 0, m.n_AC[1] = 0.

def AC_num_rule(m,i):
    return m.n_AC[1] <= 100
m.AC_num_cons = en.Constraint(rule=AC_num_rule)

In [123]:
# AC absorbs the residual heat of CHP
def AC_gas_rule(m,i):
    return m.Vgas_CHP[i] == m.Vgas_CHP_Q[i] + m.Vgas_AC[i]
m.AC_gas_cons = en.Constraint(m.Time,rule=AC_gas_rule)

In [124]:
# AC model
def AC_heat_to_cool_rule(m,i):
    return m.Q_AC[i] == m.AC_COP*m.H_gas*m.Vgas_AC[i]
m.AC_heat_to_cool_cons = en.Constraint(m.Time,rule=AC_heat_to_cool_rule)

In [125]:
# AC output constraint
def AC_rate_rule1(m,i):
    return m.Q_AC[i] <= m.Q_AC_Max*m.n_AC[1]
m.AC_rate_cons1 = en.Constraint(m.Time,rule=AC_rate_rule1)

In [126]:
# AC output constraint
def AC_rate_rule2(m,i):
    return 0 <= m.Q_AC[i]
m.AC_rate_cons2 = en.Constraint(m.Time,rule=AC_rate_rule2)

In [127]:
# define PV deployment number
m.n_PV = en.Var([1],within=en.NonNegativeIntegers)

In [128]:
# define PV output power
m.P_PV = en.Var(m.Time, within=en.Reals)

In [129]:
# Number of photovoltaic installations
# The number of installations cannot exceed the upper limit
def PV_num_rule(m):
    return m.n_PV[1] <= 200000
m.PV_num_cons = en.Constraint(m.Time, rule=PV_num_rule)

In [130]:
# Number of gas turbine installations
# The number of installations cannot exceed the upper limit
def GT_num_rule(m):
    return m.n_GT[1] <= 40
m.GT_num_cons = en.Constraint(rule=GT_num_rule)

In [131]:
# Number of CHP installations
# The number of installations cannot exceed the upper limit
def CHP_num_rule(m):
    return m.n_CHP[1] <= 40
m.CHP_num_cons = en.Constraint(rule=CHP_num_rule)

In [132]:
# If the reader wants to reproduce the experimental results of Table 7 and Table 8 in Section 5.2.
# the number of installations of CHP can be changed in this constraint
# If no CHP unit is installed, m.n_CHP[1] == 0.
# If only one CHP unit is installed, m.n_CHP[1] == 1.
# If only two CHP unit is installed, m.n_CHP[1] == 2.
# (Users can switch between the two cases at will)

# def CHP_num_rule(m):
#     return m.n_CHP[1] == 0
# m.CHP_num_cons = en.Constraint(rule=CHP_num_rule)

In [133]:
# CHP power output  
def CHP_electricity_rule(m,i):
    return m.P_CHP[i] == m.CHP_to_Electricity_Efficiency*m.Vgas_CHP[i]*m.H_gas
m.CHP_ele_cons = en.Constraint(m.Time,rule=CHP_electricity_rule)

In [134]:
# CHP heating output  
def CHP_heat_rule(m,i):
    return m.Q_CHP[i] == m.CHP_to_Heat_Efficiency*m.Vgas_CHP_Q[i]*m.H_gas
m.CHP_heat_cons = en.Constraint(m.Time,rule=CHP_heat_rule)

In [135]:
# CHP power constraints
def P_CHP_rule(m,i):
    return m.P_CHP[i] <= m.P_CHP_Max*m.n_CHP[1]
m.P_CHP_cons = en.Constraint(m.Time,rule=P_CHP_rule)

In [136]:
# CHP power constraints
def P_CHP_rule2(m,i):
    return 0 <= m.P_CHP[i]
m.P_CHP_cons2 = en.Constraint(m.Time,rule=P_CHP_rule2)

In [137]:
# CHP heating power constraints
def Q_CHP_rule(m,i):
    return m.Q_CHP[i] <= m.Q_CHP_Max*m.n_CHP[1]
m.Q_CHP_cons = en.Constraint(m.Time,rule=Q_CHP_rule)

In [138]:
# CHP heating power constraints
def Q_CHP_rule2(m,i):
    return 0 <= m.Q_CHP[i]
m.Q_CHP_cons2 = en.Constraint(m.Time,rule=Q_CHP_rule2)

In [139]:
# GT model
def GT_define_rule(m,i):
    return m.Vgas_GT[i]*m.H_gas*m.GT_to_Electricity_Efficient == m.P_GT[i]
m.GT_define_cons = en.Constraint(m.Time,rule=GT_define_rule)

In [140]:
# Gas turbine power rating
def GT_rate_rule(m,i):
    return m.P_GT[i] <= m.n_GT[1]*m.GT_Max
m.GT_rate_cons = en.Constraint(m.Time,rule=GT_rate_rule)

In [141]:
# Gas turbine power rating
def GT_rate_rule2(m,i):
    return 0 <= m.P_GT[i]
m.GT_rate2_cons = en.Constraint(m.Time,rule=GT_rate_rule2)

In [142]:
# electricity purchase
def P_buy_rule2(m,i):
    return 0 <= m.P_buy[i]
m.P_buy_cons2 = en.Constraint(m.Time, rule=P_buy_rule2)

In [143]:
# Electricity trading constraints (sell) maximum
def P_sell_rule1(m,i):
   # return m.P_sell[i] <= m.PV_unit[i] * m.n_PV[1]
    return m.P_sell[i] <= 500
m.P_sell_cons1 = en.Constraint(m.Time, rule=P_sell_rule1)

In [144]:
# Electricity trading constraints (sell)  minimum
def P_sell_rule2(m,i):
    return 0 <= m.P_sell[i]
m.P_sell_cons2 = en.Constraint(m.Time, rule=P_sell_rule2)

In [145]:
# PV power
def P_PV_rule1(m,i):
    return m.P_PV[i] == m.PV_unit[i] * m.n_PV[1]
m.P_PV_cons1 = en.Constraint(m.Time, rule=P_PV_rule1)

In [146]:
# PV output constrains minimum 
def P_PV_rule2(m,i):
    return 0 <= m.P_PV[i]
m.P_PV_cons2 = en.Constraint(m.Time, rule=P_PV_rule2)

In [147]:
# HP parameters
m.P_HP = en.Var(m.Time, within=en.Reals)
m.n_HP = en.Var([1],within=en.NonNegativeIntegers)
m.HP_COP = en.Param(initialize=2)
m.P_HP_Max = en.Param(initialize=400)

In [148]:
# heat pump power constraints maximum
def P_HP_rule1(m,i):
    return( m.P_HP[i] <= m.n_HP[1]*400 )
m.P_HP_cons1 = en.Constraint(m.Time, rule=P_HP_rule1)

In [149]:
# heat pump power constraints minimax
def P_HP_rule2(m,i):
    return( 0 <= m.P_HP[i] )
m.P_HP_cons2 = en.Constraint(m.Time, rule=P_HP_rule2)

In [150]:
# heat pump investment constraint
def num_HP_rule(m,i):
    return( m.n_HP[1] <= 30000 )
m.num_HP_cons = en.Constraint(rule=num_HP_rule)

In [151]:
# electricity trading constraint
def P_buy_rule1(m,i):
    return m.P_buy[i] <= 500
m.P_buy_cons1 = en.Constraint(m.Time, rule=P_buy_rule1)

In [152]:
# heating balance
def H_balance_rule(m,i):
    return( m.P_HP[i]*2 + m.Q_CHP[i] + m.TSS_P_dis[i] == m.Heat[i] + m.TSS_P_char[i] )
m.H_balance_cons = en.Constraint(m.Time,rule=H_balance_rule)

In [153]:
# electric energy balance
def E_balance_rule(m,i):
    return m.Load[i] + m.P_sell[i] + m.P_HP[i] + m.P_EC[i] + m.Batt_P_char[i] == m.P_PV[i] + m.P_buy[i] + m.P_GT[i] + m.P_CHP[i] + m.Batt_P_dis[i]    
m.E_balance_cons = en.Constraint(m.Time, rule=E_balance_rule)

In [154]:
# cooling balance
def C_balance_rule(m,i):
    return m.Q_EC[i] + m.Q_AC[i] == m.Cool[i]
m.C_balance_cons = en.Constraint(m.Time,rule=C_balance_rule)

In [155]:
# discount rate
r = 0.08
# service year is 25
R25 = (r*((1+r)**25))/( ((1+r)**25) - 1 ) 
# service year is 20
R20 = (r*((1+r)**20))/( ((1+r)**20) - 1 ) 
# service year is 10
R10 = (r*((1+r)**10))/( ((1+r)**10) - 1 ) 

In [156]:
# objective function
# This objective function is the case where the carbon emission penalty multiplier is 1.
# If readers want to reproduce the experimental results of Table 9 and Table 10 in Section 5.3, 
# they can change the multiplier values by themselves.
# The alternative values of multipliers are 1, 10, 20, 50, and 100.
# multiplier = 10
# multiplier = 20
# multiplier = 50
# multiplier = 100
multiplier = 1
def Obj_fn(m):
    return ( sum( (m.priceBuy[i]*m.P_buy[i]) - (m.priceSell[i]*m.P_sell[i]) + (multiplier*5*0.00089*m.P_buy[i]) + (multiplier*5*0.0001862*m.H_gas*(m.Vgas_GT[i]+m.Vgas_CHP[i])) + (m.priceGas[i]*(m.Vgas_GT[i]+m.Vgas_CHP[i])) for i in m.Time ) + R25*(198)*m.n_PV[1] + R25*70572*m.n_HP[1] + R25*203808*m.n_GT[1] + R25*505766*m.n_CHP[1] + R25*36438*m.n_EC[1] + R25*27483*m.n_AC[1] + R10*444*m.n_ESS[1] + R20*1400*m.n_TSS[1] + 0.03*198*m.n_PV[1] + 0.03*70572*m.n_HP[1] + 0.03*203808*m.n_GT[1] + 0.03*505766*m.n_CHP[1] + 0.03*36438*m.n_EC[1] + 0.03*27483*m.n_AC[1] + 0.03*444*m.n_ESS[1] + 0.03*1400*m.n_TSS[1] )                
m.total_cost = en.Objective(rule=Obj_fn, sense=en.minimize)

In [157]:
# Set the path to the solver
# Specify your own path to cplex or any other solver
opt = SolverFactory("cplex", executable="C:/Program Files/IBM/ILOG/CPLEX_Studio128/cplex/bin/x64_win64/cplex")

In [158]:
results = opt.solve(m)

In [159]:
print(results)


Problem: 
- Name: tmptbnu7hpq
  Lower bound: 1483850.3767
  Upper bound: 1483991.2346239286
  Number of objectives: 1
  Number of constraints: 543126
  Number of variables: 245289
  Number of nonzeros: 1116634
  Sense: minimize
Solver: 
- Status: ok
  User time: 2135.13
  Termination condition: optimal
  Termination message: MIP - Integer optimal, tolerance (0.0001/1e-06)\x3a Objective = 1.4839912346e+06
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 11744
      Number of created subproblems: 11744
  Error rc: 0
  Time: 2139.3299703598022
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [160]:
P_sell = np.zeros((len(sellPrice),1))
for i in m.Time:
    P_sell[i] = en.value(m.P_sell[i])

In [161]:
P_buy = np.zeros((len(sellPrice),1))
for i in m.Time:
    P_buy[i] = en.value(m.P_buy[i])

In [162]:
P_PV = np.zeros((len(sellPrice),1))
for i in m.Time:
    P_PV[i] = en.value(m.P_PV[i])

In [163]:
# The number of PV configured in the microgrid
n_PV = en.value(m.n_PV[1])
print(n_PV)

7239.0


In [164]:
# The number of HP configured in the microgrid
n_HP = en.value(m.n_HP[1])
print(n_HP)

8.0


In [165]:
# The number of GT configured in the microgrid
n_GT = en.value(m.n_GT[1])
print(n_GT)

7.0


In [166]:
# The number of CHP configured in the microgrid
n_CHP = en.value(m.n_CHP[1])
print(n_CHP)

3.0


In [167]:
# The number of EC configured in the microgrid
n_EC = en.value(m.n_EC[1])
print(n_EC)

0.9999999999972428


In [168]:
# The number of AC configured in the microgrid
n_AC = en.value(m.n_AC[1])
print(n_AC)

4.0


In [169]:
# The number of ESS configured in the microgrid
n_ESS = en.value(m.n_ESS[1])
print(n_ESS)

0.0


In [170]:
# The number of TSS configured in the microgrid
n_TSS = en.value(m.n_TSS[1])
print(n_TSS)

52.0


In [171]:
# Output other optimization variables
Q_AC = np.zeros((len(sellPrice),1))
for i in m.Time:
    Q_AC[i] = en.value(m.Q_AC[i])
    
P_GT = np.zeros((len(sellPrice),1))
for i in m.Time:
    P_GT[i] = en.value(m.P_GT[i])
    
P_CHP = np.zeros((len(sellPrice),1))
for i in m.Time:
    P_CHP[i] = en.value(m.P_CHP[i])
    
Q_CHP = np.zeros((len(sellPrice),1))
for i in m.Time:
    Q_CHP[i] = en.value(m.Q_CHP[i])
    
P_HP = np.zeros((len(sellPrice),1))
for i in m.Time:
    P_HP[i] = en.value(m.P_HP[i])
    
P_EC = np.zeros((len(sellPrice),1))
for i in m.Time:
    P_EC[i] = en.value(m.P_EC[i])
    
TSS_P_dis = np.zeros((len(sellPrice),1))
for i in m.Time:
    TSS_P_dis[i] = en.value(m.TSS_P_dis[i])
    
TSS_P_char = np.zeros((len(sellPrice),1))
for i in m.Time:
    TSS_P_char[i] = en.value(m.TSS_P_char[i])
    
P_buy = np.zeros((len(sellPrice),1))
for i in m.Time:
    P_buy[i] = en.value(m.P_buy[i])
    
Q_EC = P_EC*4
Q_HP = P_HP*2

In [182]:
# Output the cost of purchasing and selling electricity
C_grid = sum( (buyPrice[i]*P_buy[i]) - (sellPrice[i]*P_sell[i]) for i in range(8760) )
C_grid_heating1 = sum( (buyPrice[i]*P_buy[i]) - (sellPrice[i]*P_sell[i]) for i in range(0,3624) ) 
C_grid_heating2 = sum( (buyPrice[i]*P_buy[i]) - (sellPrice[i]*P_sell[i]) for i in range(6552,8760) ) 
C_grid_heating = C_grid_heating1 + C_grid_heating2
C_grid_cooling = sum( (buyPrice[i]*P_buy[i]) - (sellPrice[i]*P_sell[i]) for i in range(3624,6552) ) 
print(C_grid)

[170494.58917108]


In [183]:
# Table 8, Electricity Period I 
print(C_grid_heating)

[118644.0104409]


In [184]:
# Table 8, Electricity Period II
print(C_grid_cooling)

[51850.57873018]


In [185]:
# Output investment cost
C_inv = R25*198*n_PV + R25*70572*n_HP + R25*203808*n_GT + R25*505766*n_CHP + R25*36438*n_EC + R25*27483*n_AC + R10*444*n_ESS + R20*1400*n_TSS        
print(C_inv)

484073.26530767925


In [186]:
# Output the cost of purchasing gas
Vgas_GT = np.zeros((len(sellPrice),1))
for i in m.Time:
    Vgas_GT[i] = en.value(m.Vgas_GT[i])
    
Vgas_CHP = np.zeros((len(sellPrice),1))
for i in m.Time:
    Vgas_CHP[i] = en.value(m.Vgas_CHP[i])

C_gas = sum( (price_gas[i]*(Vgas_GT[i]+Vgas_CHP[i])) for i in range(8760))
C_gas_heating1 = sum( (price_gas[i]*(Vgas_GT[i]+Vgas_CHP[i])) for i in range(0,3624))
C_gas_heating2 = sum( (price_gas[i]*(Vgas_GT[i]+Vgas_CHP[i])) for i in range(6552,8760))
C_gas_heating = C_gas_heating1 + C_gas_heating2
C_gas_cooling = sum( (price_gas[i]*(Vgas_GT[i]+Vgas_CHP[i])) for i in range(3624,6552)) 

print(C_gas)

[641265.24289371]


In [187]:
# Table 8, Gas Period I 
print(C_gas_heating)

[531076.71774078]


In [188]:
# Table 8, Gas Period II 
print(C_gas_cooling)

[110188.52515294]


In [189]:
# Output Maintenance cost
C_maintain = 0.03*198*n_PV + 0.03*70572*n_HP + 0.03*203808*n_GT + 0.03*505766*n_CHP + 0.03*36438*n_EC + 0.03*27483*n_AC + 0.03*444*n_ESS + 0.03*1400*n_TSS    
print(C_maintain)

154830.65999999698


In [190]:
# Output CO2 emission penalizes the cost
# Note that the carbon emission penalty multiplier should be consistent with the objective function
C_emission = sum( (multiplier*5*0.00089*P_buy[i]) + (multiplier*5*0.0001862*9.97*(Vgas_GT[i]+Vgas_CHP[i])) for i in range(8760))
C_emission_heating1=sum( (multiplier*5*0.00089*P_buy[i])+(multiplier*5*0.0001862*9.97*(Vgas_GT[i]+Vgas_CHP[i])) for i in range(0,3624))
C_emission_heating2=sum( (multiplier*5*0.00089*P_buy[i])+(multiplier*5*0.0001862*9.97*(Vgas_GT[i]+Vgas_CHP[i])) for i in range(6552,8760))
C_emission_heating = C_emission_heating1 + C_emission_heating2
C_emission_cooling = sum( (multiplier*5*0.00089*P_buy[i]) + (multiplier*5*0.0001862*9.97*(Vgas_GT[i]+Vgas_CHP[i])) for i in range(3625,6552))
print(C_emission)

[33327.4772515]


In [194]:
# Table 8, Emission Period I 
print(C_emission_heating)

[26110.38620931]


In [195]:
# Table 8, Emission Period II 
print(C_emission_cooling)

[7214.57958065]


In [196]:
# Output annualized total cost
C_total = C_emission + C_maintain + C_gas + C_inv + C_grid
print(C_total)

[1483991.23462396]
